# SQL V: Exercise 2 solution 

To solve this exercise you'll use the database `classicmodels`

In [1]:
#Imports
import pandas as pd
import mysql.connector

#SQL Function
def sql(_query, _con):
    
    cursor = _con.cursor()
    cursor.execute(_query)
    
    cols = cursor.column_names
    data = cursor.fetchall()
    
    _con.commit()
    
    df = pd.DataFrame(data, columns=cols)
    
    return df

#Connections
dw = mysql.connector.connect(
  host="iosqlde.onairnet.xyz",
  user="assemblerReader",
  password="!Reader2022",
  database="classicmodels"
)

In [2]:
# 1. Get a list of customers that ordered the prodcut "1938 Cadillac V-16 Presidential Limousine"

query = """
SELECT
customers.customerName
FROM (
	SELECT 
		orderdetails.orderNumber
	FROM products

	LEFT JOIN orderdetails ON
		products.productCode = orderdetails.productCode

	WHERE productName = '1938 Cadillac V-16 Presidential Limousine'
) base

LEFT JOIN orders ON
base.orderNumber = orders.orderNumber

LEFT JOIN customers ON
orders.customerNumber = customers.customerNumber
"""
sql(query, dw)

,customerName
0,"Blauer See Auto, Co."
1,"AV Stores, Co."
2,Signal Gift Stores
3,Kelly's Gift Shop
4,Signal Collectibles Ltd.
5,Corporate Gift Ideas Co.
6,Rovelli Gifts
7,Mini Gifts Distributors Ltd.
8,"Australian Collectables, Ltd"
9,Euro+ Shopping Channel


In [4]:
#2. Get the most sold proctuct name for each employee. Shown the employee name and surname only in a single field + the proct name in another field + the amount sold

query = """
SELECT 
employeeNumber as employeeID,
CONCAT(firstName, ' ', lastName) as employeeName,
SUBSTRING_INDEX(MAX(CONCAT(orderdetails.quantityOrdered, '#@#', products.productName)), '#@#', -1) as productName,
SUBSTRING_INDEX(MAX(CONCAT(orderdetails.quantityOrdered, '#@#', products.productName)), '#@#', 1) as productAmount

FROM employees

LEFT JOIN customers ON
employees.employeeNumber = customers.salesRepEmployeeNumber

LEFT JOIN orders ON
orders.customerNumber = customers.customerNumber

LEFT JOIN orderdetails ON
orders.orderNumber = orderdetails.orderNumber

LEFT JOIN products ON
orderdetails.productCode = products.productCode

GROUP BY employeeID, employeeName

ORDER BY employeeName
"""
sql(query, dw)

,employeeID,employeeName,productName,productAmount
0,1611,Andy Fixter,1932 Alfa Romeo 8C2300 Spider Sport,66
1,1143,Anthony Bow,None,None
2,1504,Barry Jones,1974 Ducati 350 Mk3 Desmo,50
3,1002,Diane Murphy,None,None
4,1286,Foon Yue Tseng,The Queen Mary,50
5,1323,George Vanauf,America West Airlines B757-200,85
6,1102,Gerard Bondur,None,None
7,1370,Gerard Hernandez,1969 Dodge Charger,97
8,1076,Jeff Firrelli,None,None
9,1188,Julie Firrelli,1968 Ford Mustang,50


In [5]:
#3. Get for each customer, the amount spend, the amount paid and the amount pending to be paid

query = """
SELECT
result.customerName,
MAX(amountOrdered),
SUM(amount) as amountPaid,
MAX(amountOrdered)- SUM(amount) as amountPending
FROM (

	SELECT
		customerName,
		MAX(customerNumber) as customerID,
		SUM(amountOrdered) as amountOrdered
	FROM (
		SELECT
			customerName,
			customers.customerNumber,
			orders.orderNumber,
			orderdetails.quantityOrdered * orderdetails.priceEach as amountOrdered

		FROM customers

		LEFT JOIN orders ON
		orders.customerNumber = customers.CustomerNumber

		LEFT JOIN orderdetails ON
		orders.orderNumber = orderdetails.orderNumber

		WHERE orders.status <> 'Cancelled'
	)base

	GROUP BY customerName

) result

LEFT JOIN payments ON
result.customerID = payments.customerNumber

GROUP BY customerName
"""
sql(query, dw)

,customerName,MAX(amountOrdered),amountPaid,amountPending
0,Online Diecast Creations Co.,116449.29,116449.29,0.00
1,"Blauer See Auto, Co.",75937.76,75937.76,0.00
2,Vitachrome Inc.,72497.64,72497.64,0.00
3,Baane Mini Imports,104224.79,104224.79,0.00
4,Euro+ Shopping Channel,773624.18,715738.98,57885.20
...,...,...,...,...
93,"Norway Gifts By Mail, Co.",69059.04,69059.04,0.00
94,Marta's Replicas Co.,90545.37,90545.37,0.00
95,Scandinavian Gift Ideas,76776.44,76776.44,0.00
96,"Bavarian Collectables Imports, Co.",31310.09,31310.09,0.00
